In [21]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertAdam, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io, os, sys
import numpy as np
from sklearn.metrics import accuracy_score

In [24]:
# initializations
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
print(n_gpu, torch.cuda.get_device_name(0))
MAX_LEN = 128
# Select a batch size for training. For fine-tuning BERT on a specific task, it is recommend a batch size of 16 or 32
batch_size = 128
# Number of training epochs (it is recommend between 2 and 4 - we dont want to overtrain, just fine tune the model)
epochs = 8
num_labels = 2

2 Tesla V100-PCIE-32GB


In [25]:
class BertDataPreprocessorLoader:
    def __init__(self, dataframe_path):
        tokenizer_path = '../bertPytorch/bert-base-cased'
        assert os.path.exists(tokenizer_path)
        tokenizer = BertTokenizer.from_pretrained(tokenizer_path)
               
        df = pd.read_csv(dataframe_path, usecols=['text', 'truth', 'split'])
        train_df = df[df['split']=='TRAIN']
        test_df = df[df['split']=='TEST']
        train_labels = train_df.truth.values
        test_labels = test_df.truth.values
        
        train_valid_inputs, train_valid_masks = self.preprocess_for_BERT(train_df, tokenizer)       
        train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_valid_inputs, train_labels, random_state=2018, test_size=0.1)
        train_masks, validation_masks, _, _ = train_test_split(train_valid_masks, train_valid_inputs, random_state=2018, test_size=0.1)
        print(train_inputs.shape, validation_inputs.shape, train_labels.shape, validation_labels.shape)
        
        test_inputs, test_masks = self.preprocess_for_BERT(test_df, tokenizer)
        print(test_inputs.shape, test_labels.shape)
        
        train_inputs = torch.tensor(train_inputs)
        validation_inputs = torch.tensor(validation_inputs)
        test_inputs = torch.tensor(test_inputs)
        train_labels = torch.tensor(train_labels)
        validation_labels = torch.tensor(validation_labels)
        test_labels = torch.tensor(test_labels)
        train_masks = torch.tensor(train_masks)
        validation_masks = torch.tensor(validation_masks)
        test_masks = torch.tensor(test_masks)
        
        train_data = TensorDataset(train_inputs, train_masks, train_labels)
        train_sampler = RandomSampler(train_data)
        self.train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
        
        validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
        validation_sampler = SequentialSampler(validation_data)
        self.validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)
        
        test_data = TensorDataset(test_inputs, test_masks, test_labels)
        test_sampler = RandomSampler(test_data)
        self.test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)
            
    def preprocess_for_BERT(self, df, tokenizer):
        sentences = df['text'].values
        sentences = ["[CLS] " + sentence + " [SEP]" for sentence in sentences]
        tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
        input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
        input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
        
        attention_masks = []
        for seq in input_ids:
            seq_mask = [float(i>0) for i in seq]
            attention_masks.append(seq_mask)
        return input_ids, attention_masks        

In [26]:
class BertVictimModel:
    def __init__(self):
        pretrained_model_path = '../bertPytorch/bert-base-cased'
        assert os.path.exists(pretrained_model_path)
        
        self.model = BertForSequenceClassification.from_pretrained(pretrained_model_path, num_labels=num_labels)
        self.model.cuda()
        
        param_optimizer = list(self.model.named_parameters())
        no_decay = ['bias', 'gamma', 'beta']
        optimizer_grouped_parameters = [
            {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},
            {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.0}
        ]
        
        self.optimizer = BertAdam(optimizer_grouped_parameters, lr=2e-5, warmup=.1)
        
    def flat_accuracy(self, preds, labels):
        pred_flat = np.argmax(preds, axis=1).flatten()
        labels_flat = labels.flatten()
        return np.sum(pred_flat == labels_flat) / len(labels_flat)
    
    def save_model(self):
        torch.save(self.model, "bert_model_128_128_8.pth")
        
    def load_model(self):
        self.model = torch.load("bert_model_128_128_8.pth")
        
    def fit(self, train_dataloader, validation_dataloader):
        train_loss_set = [] # Store our loss and accuracy for plotting
        for _ in trange(epochs, desc="Epoch"): # trange is a tqdm wrapper around the normal python range
            ########### training
            self.model.train() # Set our model to training mode (as opposed to evaluation mode)
            tr_loss, nb_tr_examples, nb_tr_steps = 0, 0, 0  # Tracking variables

            for step, batch in enumerate(train_dataloader): # Train the data for one epoch
                print('#', end='')
                b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch) # Add batch to GPU and Unpack the inputs from our dataloader
    
                self.optimizer.zero_grad() # Clear out the gradients (by default they accumulate)
                loss = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels) # Forward pass
                train_loss_set.append(loss.item())    
                loss.backward() # Backward pass
                self.optimizer.step() # Update parameters and take a step using the computed gradient
                
                # Update tracking variables
                tr_loss += loss.item()
                nb_tr_examples += b_input_ids.size(0)
                nb_tr_steps += 1
                
                if step%100==0:
                    print(step)
            
            print("Train loss: {}".format(tr_loss/nb_tr_steps))
            
            ########### Validation
            self.model.eval() # Put model in evaluation mode to evaluate loss on the validation set
            eval_loss, eval_accuracy, nb_eval_steps, nb_eval_examples = 0, 0, 0, 0 # Tracking variables 

            for batch in validation_dataloader: # Evaluate data for one epoch
                # Add batch to GPU and Unpack the inputs from our dataloader
                b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
                
                with torch.no_grad(): # Telling the model not to compute or store gradients, saving memory and speeding up validation
                    logits = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask) # Forward pass, calculate logit predictions

                # Move logits and labels to CPU and compute accuracy
                logits = logits.detach().cpu().numpy()
                label_ids = b_labels.to('cpu').numpy()
                eval_accuracy += self.flat_accuracy(logits, label_ids)
                
                nb_eval_steps += 1 # update tracking var
                
            print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
            
    def predict(self, test_dataloader):
        self.model.eval()
        predictions , true_labels = [], []

        for batch in test_dataloader:
            # Add batch to GPU and Unpack the inputs from our dataloader
            b_input_ids, b_input_mask, b_labels = tuple(t.to(device) for t in batch)
            
            with torch.no_grad(): # Telling the model not to compute or store gradients, saving memory and speeding up prediction
                logits = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask) # Forward pass, calculate logit predictions
            
            # Move logits and labels to CPU
            logits = logits.detach().cpu().numpy()
            label_ids = b_labels.to('cpu').numpy()

            # Store predictions and true labels
            predictions.append(logits)
            true_labels.append(label_ids)
            print('#', end='')
        
        print()
        y_true = []
        y_pred = []
        for i in range(len(true_labels)):
            y_true.extend(true_labels[i])
            y_pred.extend(np.argmax(predictions[i], axis=1).flatten())
        assert len(y_true) == len(y_pred)
        return y_true, y_pred
    
    def evaluate(self, test_dataloader, metric=accuracy_score):
        y_true, y_pred = self.predict(test_dataloader)
        score = metric(y_true, y_pred)
        print(metric.__name__, score)

# Victim model training

In [27]:
data_loader = BertDataPreprocessorLoader('combined_relevant_data.csv')  

(29926, 128) (3326, 128) (29926,) (3326,)
(7126, 128) (7126,)


In [28]:
model = BertVictimModel()

t_total value of -1 results in schedule not being applied


In [29]:
model.fit(data_loader.train_dataloader, data_loader.validation_dataloader)

Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

#0
####################################################################################################100
####################################################################################################200
#################################Train loss: 0.6148226508217999


Epoch:  12%|█▎        | 1/8 [04:03<28:23, 243.34s/it]

Validation Accuracy: 0.730325663919414
#0
####################################################################################################100
####################################################################################################200
#################################Train loss: 0.41186761703246677


Epoch:  25%|██▌       | 2/8 [08:06<24:19, 243.31s/it]

...

Validation Accuracy: 0.9113057081807081
#0
####################################################################################################100
####################################################################################################200
#################################Train loss: 0.025075112296363864


Epoch:  88%|████████▊ | 7/8 [28:22<04:03, 243.21s/it]

Validation Accuracy: 0.9227192078754579
#0
####################################################################################################100
####################################################################################################200
#################################Train loss: 0.021556579534752436


Epoch: 100%|██████████| 8/8 [32:25<00:00, 243.20s/it]

Validation Accuracy: 0.9163995726495726


In [30]:
model.save_model()

# Victim model performance on test data

In [31]:
model.load_model()

In [32]:
y_true, y_pred = model.predict(data_loader.test_dataloader)

########################################################


In [33]:
print(accuracy_score(y_true, y_pred))

0.921274207128824


# Victim model performance on test data with adversarial samples mixed in

In [34]:
data_loader = BertDataPreprocessorLoader('combined_original_adv_data.csv')  
model.load_model()
y_true, y_pred = model.predict(data_loader.test_dataloader)
print(accuracy_score(y_true, y_pred))

(29926, 128) (3326, 128) (29926,) (3326,)
(20904, 128) (20904,)
####################################################################################################################################################################
0.8474454649827784


# Adversarial training

In [35]:
data_loader = BertDataPreprocessorLoader('combined_adv_training_data.csv')  
model = BertVictimModel()
model.fit(data_loader.train_dataloader, data_loader.validation_dataloader)
model.save_model()

(36126, 128) (4015, 128) (36126,) (4015,)
(14015, 128) (14015,)


t_total value of -1 results in schedule not being applied
Epoch:   0%|          | 0/8 [00:00<?, ?it/s]

#0
####################################################################################################100
####################################################################################################200
##################################################################################Train loss: 0.5527713370407428


Epoch:  12%|█▎        | 1/8 [04:53<34:14, 293.56s/it]

Validation Accuracy: 0.8068016539228724
#0
####################################################################################################100
####################################################################################################200
##################################################################################Train loss: 0.2557343966179517


...

Validation Accuracy: 0.9371467752659575
#0
####################################################################################################100
####################################################################################################200
##################################################################################Train loss: 0.019415330920078834


Epoch:  88%|████████▊ | 7/8 [34:15<04:53, 293.62s/it]

Validation Accuracy: 0.9351936502659575
#0
####################################################################################################100
####################################################################################################200
##################################################################################Train loss: 0.015548673731747146


Epoch: 100%|██████████| 8/8 [39:08<00:00, 293.60s/it]

Validation Accuracy: 0.94140625


# Victim model performance on test data with adversarial samples mixed in

In [36]:
data_loader = BertDataPreprocessorLoader('combined_adv_training_data.csv')  
model.load_model()
y_true, y_pred = model.predict(data_loader.test_dataloader)
print(accuracy_score(y_true, y_pred))

(36126, 128) (4015, 128) (36126,) (4015,)
(14015, 128) (14015,)
##############################################################################################################
0.9242240456653585
